In [1]:
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import path
import xmltodict
from myfunctions import event_separation , image_crop, image_crop_negative, poi
from myfunctions import delete_old_extracted_events
from database.extract_yaml import get_dict
import tifffile
from pathlib import Path
import copy

In [2]:
BASE_DIR = r'C:\Users\roumba\Documents\Software\deep-events'
SAVING_SCHEME = "ws0.1"

## Input data from a folder ##

In [3]:
files_dir = os.path.join(BASE_DIR, 'original_data')
images_dir = '221214_allbrightfield_cos7_ZEISS_bf'
joined_path = os.path.join(files_dir, images_dir)
size=(2048,2048)
scaling_ZEISS=0.103
scaling_iSIM=0.056
img,input_name,output_name,datacsv,pixel_size={},{},{},{},{}

path = Path(joined_path)
files = list(path.glob("*.ome.tif"))
files.extend(list(path.glob("*.csv")))

for input_file in files:
    input_file = str(input_file.name)
    joined_file_path = os.path.join(files_dir, images_dir, input_file)
    if '.tif' in input_file:
        tif  = tifffile.TiffFile(joined_file_path)
        mdInfoDict = xmltodict.parse(tif.ome_metadata, force_list={'Plane'})
        date, cell_type, dye, bf_fl, index  = input_file.split('_')
        number, ome, tiff= index.split('.')
        img[int(number)-1] = Image.open(joined_file_path)
        input_name[int(number)-1]= f'points_{date}_{cell_type}_{bf_fl}_{number}'
        output_name[int(number)-1]= f'image_{date}_{cell_type}_{bf_fl}_{number}'
        print('Loaded image:'+input_file)
    else:
        labels, index, bf_fl  = input_file.split('_')
        datacsv[int(index)-1] = pd.read_csv(joined_file_path)
        
        if (scaling_iSIM*size[0]) < datacsv['axis-1'].max() < (scaling_ZEISS*size[0]):
            pixel_size[int(number)-1] = scaling_ZEISS

        elif datacsv['axis-1'].max() < (scaling_iSIM*size[0]):
            pixel_size[int(number)-1] = scaling_iSIM
            
        else:
            pixel_size[int(number)-1] = 1

        print('Loaded csv:'+ input_file)
print('Pixel scaling:',pixel_size)
max_number=int(len(datacsv))   

Loaded image:220915_cos7_mitostaygold_Brightfield_1.ome.tif
Loaded image:220915_cos7_mitostaygold_Brightfield_2.ome.tif
Loaded image:220915_cos7_mitostaygold_Brightfield_3.ome.tif
Loaded image:221123_cos7_mitostaygold_Brightfield_4.ome.tif
Loaded image:221123_cos7_mitostaygold_Brightfield_5.ome.tif
Loaded csv:labels_1_bf.csv
Loaded csv:labels_2_bf.csv
Loaded csv:labels_3_bf.csv
Loaded csv:labels_4_bf.csv
Loaded csv:labels_5_bf.csv
Pixel scaling: {}


## Get all Gaussian Points of Interest ##

In [4]:
for x in range(max_number):
    csv=datacsv[x]
    im=img[x]

    scale_value=float(pixel_size[x])
    print('scaling by',scale_value)
    print(csv['axis-1'][1])
    csv['axis-1']=csv['axis-1']*(1/scale_value)
    print(csv['axis-1'][1])
    csv['axis-2']=csv['axis-2']*(1/scale_value)
    
    for i in range(2,7,1):
        sigma=(i,i)
        s=sigma[0]
        in_name=f'{input_name[x]}_sigma{s}.tiff'
        framenum=im.n_frames
        poi(csv,in_name,sigma,size,framenum)

100%|██████████| 21/21 [00:05<00:00,  3.79it/s]


## Cut Images and Gaussian POIs ##

In [5]:
base_dir = BASE_DIR
training_dir = os.path.join(base_dir, 'training_data')
folder_name = images_dir+'_pos'
filepath= os.path.join(training_dir , folder_name)


for x in range(max_number):
    
    csv=datacsv[x]
    im=img[x]
    print('file number:',x+1)
    list1=event_separation(csv)
    l=len(list1)
    out_name=f'{output_name[x]}'
    event_dict = image_crop(l,list1, csv, im,0, out_name,filepath)

    for i in range(2,7,1):
        sigma=(i,i)
        s=sigma[0]
        print('sigma:',s)
        in_name=f'{input_name[x]}_sigma{s}.tiff'
        out_name_g=f'{output_name[x]}_sigma{s}'
        
        gauss=base_dir+f'\{in_name}'
        gauss_image=Image.open(gauss)
        image_crop(l,list1, csv, gauss_image,1, out_name_g,filepath)

file number: 1
sigma: 2
sigma: 3
sigma: 4
sigma: 5
sigma: 6
file number: 2
sigma: 2
sigma: 3
sigma: 4
sigma: 5
sigma: 6
file number: 3
sigma: 2
sigma: 3
sigma: 4
sigma: 5
sigma: 6
file number: 4
sigma: 2
sigma: 3
sigma: 4
sigma: 5
sigma: 6
file number: 5
sigma: 2
sigma: 3
sigma: 4
sigma: 5
sigma: 6


## Cut images and gaussians DATABASE style

In [8]:
base_dir = BASE_DIR
training_dir = os.path.join(base_dir, 'training_data')
if "ws" in SAVING_SCHEME:
    folder_name = images_dir
else:
    folder_name=images_dir+'_pos'
filepath= os.path.join(training_dir , folder_name)

tif_files = list(item for item in files if ".tif" in item.name)
folder_dict = get_dict(images_dir,os.path.join(BASE_DIR, "original_data"))
delete_old_extracted_events(folder_dict, os.path.join(training_dir, images_dir))
event_dict = copy.deepcopy(folder_dict)
event_dict['type'] = "event"
del event_dict['extracted_events']

for x in range(max_number):
    event_dict['original_file'] = os.path.basename(tif_files[x])
    event_dict['event_content'] = 'division'
    
    csv=datacsv[x]
    im=img[x]
    print('file number:',x+1)
    ##
    # scale_value=float(pixel_size[x])
    # print('scaling by',scale_value)
    # print(csv['axis-1'][1])
    # csv['axis-1']=csv['axis-1']*(1/scale_value)
    # print(csv['axis-1'][1])
    # csv['axis-2']=csv['axis-2']*(1/scale_value)
    ##
    list1=event_separation(csv)
    l=len(list1)
    for event in list1:
        event = [event]
        out_name=f'{output_name[x]}'
        event_dict = image_crop(1,event, csv, im,0, out_name,filepath, SAVING_SCHEME=SAVING_SCHEME,
                folder_dict=folder_dict, event_dict=event_dict)

        for i in range(6,7,1):
            sigma=(i,i)
            s=sigma[0]
            print('sigma:',s)
            in_name=f'{input_name[x]}_sigma{s}.tiff'
            out_name_g=f'{output_name[x]}_sigma{s}'
            
            gauss=base_dir+f'\{in_name}'
            gauss_image=Image.open(gauss)
            image_crop(1,event, csv, gauss_image,1, out_name,filepath, SAVING_SCHEME=SAVING_SCHEME,
                folder_dict=folder_dict, event_dict=event_dict)

file number: 1
[0, 1, 2, 3, 4, 5, 6, 7]
sigma: 6
[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
sigma: 6
[20, 21, 22, 23, 24, 25]
sigma: 6
[26, 27, 28]
sigma: 6
[29, 30, 31, 32, 33]
sigma: 6
[34, 35, 36, 37]
sigma: 6
[38, 39, 40, 41]
sigma: 6
file number: 2
[0, 1, 2, 3, 4, 5]
sigma: 6
[6, 7, 8, 9]
sigma: 6
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
sigma: 6
[25, 26, 27]
sigma: 6
[28, 29, 30, 31, 32, 33, 34, 35]
sigma: 6
[36, 37, 38, 39]
sigma: 6
[40, 41, 42, 43, 44, 45]
sigma: 6
[46, 47, 48]
sigma: 6
[49, 50]
sigma: 6
[51, 52, 53, 54, 55, 56]
sigma: 6
[57, 58, 59, 60, 61, 62]
sigma: 6
[63, 64, 65, 66, 67]
sigma: 6
[68, 69, 70, 71, 72]
sigma: 6
[73, 74, 75, 76, 77, 78, 79, 80, 81, 82]
sigma: 6
[83, 84, 85, 86]
sigma: 6
[87, 88, 89, 90, 91, 92, 93, 94, 95]
sigma: 6
[96, 97, 98, 99, 100, 101]
sigma: 6
[102, 103, 104, 105, 106, 107, 108]
sigma: 6
[109, 110, 111, 112, 113, 114, 115, 116, 117]
sigma: 6
[118, 119, 120, 121, 122, 123, 124]
sigma: 6
[125, 126, 127, 128, 129, 1

## Cut Negative Frames ##

In [6]:
base_dir= r'C:\Users\roumba\Documents\Software\deep-events'
training_dir= r'C:\Users\roumba\Documents\Software\deep-events\training_data'
folder_name=images_dir+'_neg'
filepath= os.path.join(training_dir , folder_name)
from myfunctions import image_crop_negative

for x in range(max_number):
    csv=datacsv[x]
    im=img[x]
    print('file number:',x+1)
    out_name=f'{output_name[x]}'

    list1=event_separation(csv)
    l=len(list1)
    image_crop_negative(l,list1, csv, im, out_name,filepath)

file number: 1
file number: 2
file number: 3
file number: 4
file number: 5
